In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from reinforce_learning import Agent
from dataset import Dataset
from features import StrategiesSignals

In [24]:
TICKER = "BTCUSDT"
data_binance = Dataset()
data = data_binance.get_data(days=90, ticker=TICKER, ts='1H')

In [25]:
data['reward'] = data['close'].pct_change().shift(-1)

# Create Features

## Mean reversing sigma

In [26]:
# We need take best rules
data['mean_reversing_sigma'] = StrategiesSignals.mean_reversing_sigma(
        data=data,
        sigma=1,
        lag=1,
        window_ma=100,
        window_std=100
    )

## Mean Reversing

In [35]:
data['mean_reversin'] = StrategiesSignals.mean_reversing(data=data, window=200)

## Momentum

In [33]:
data['momentum'] = StrategiesSignals.momentum(df=data['close'].values, seq=2)

In [38]:
# What is min max scaller?
scaler = MinMaxScaler()
scaler.fit(data.drop(['reward'], axis=1).values)

MinMaxScaler()

In [39]:
train = pd.DataFrame(
    scaler.transform(data
                        .drop(['reward'], axis=1)
                        .values)
)
train.head(10)

,0,1,2,3,4,5,6,7,8,9
0,0.829607,0.850481,0.837382,0.842639,0.100940,0.5,0.0,0.5,0.5,0.5
1,0.842639,0.860587,0.852611,0.862780,0.066918,0.5,0.0,0.5,0.5,0.5
2,0.862780,0.862998,0.876243,0.862873,0.040025,0.5,0.0,0.5,1.0,0.5
3,0.862874,0.867858,0.877177,0.872315,0.030473,0.5,0.0,0.5,0.5,0.5
4,0.872315,0.872158,0.865195,0.853871,0.054981,0.5,0.0,0.5,0.5,0.5
5,0.853871,0.850434,0.846120,0.830740,0.074943,0.5,0.0,0.5,0.0,0.5
6,0.830924,0.833750,0.829084,0.823687,0.111722,0.5,0.0,0.5,0.5,0.5
7,0.823745,0.816398,0.795012,0.793517,0.200965,0.5,0.0,0.5,0.5,0.5
8,0.793517,0.802993,0.802217,0.792542,0.112344,0.5,0.0,0.5,0.5,0.5
9,0.792543,0.795420,0.717620,0.735053,0.364711,0.5,0.0,0.5,0.5,0.5


In [40]:
train.loc[:, 'time'] = data.index
train.loc[:, 'open_price'] = data['close'].values
train.loc[:, 'close_price'] = data['close'].shift(-1).values
train.loc[:, 'reward'] = train['close_price'] - train['open_price']
train.head(2)

,0,1,2,3,4,5,6,7,8,9,time,open_price,close_price,reward
0,0.829607,0.850481,0.837382,0.842639,0.100940,0.5,0.0,0.5,0.5,0.5,2021-03-15 01:00:00,59404.48,60066.56,662.08
1,0.842639,0.860587,0.852611,0.862780,0.066918,0.5,0.0,0.5,0.5,0.5,2021-03-15 02:00:00,60066.56,60069.63,3.07


In [41]:
BATCH_SIZE = 200 # bach size - for neural network. In important to learn RF
MEMORY_SIZE = 2000 # Experiece 
ACTION_SPACE = [0, 1, 2]  # short, hold, buy
EPISODES = 20 # Hiw many times we lear RF
X_VAR = train.drop(['reward', 'open_price', 'close_price', 'time'], axis=1).columns
WINDOW = len(X_VAR)
Y_VAR = ['reward']
ARCITECTURE = (256, 256)
horizon = '1'
trader = Agent(
    name='{}_hour_bitcoin_test'.format(horizon),
    window=0,
    type_model="trader_1",  # forward_net, trader_0, trader_1
    data=train.dropna().head(1000),
    window_size=WINDOW,
    batch_size=BATCH_SIZE,
    action_space=ACTION_SPACE,
    episodes=EPISODES,
    memory_size=MEMORY_SIZE,
    X_var=X_VAR,
    Y_var=Y_VAR,
    architecture=ARCITECTURE
)
trader.train(load_model=False)

 32%|███▏      | 315/999 [00:16<00:35, 19.48it/s]


KeyboardInterrupt: 

In [23]:
print(X_VAR)

Index([0, 1, 2, 3, 4, 5], dtype='object')


In [12]:
test = train.dropna().head(100)